# Lab 6a
## The Final Lab!

I know it seems like only yesterday we were first writing "import arcpy," but here we are. You've learned a lot along the way and now, in this final lab, I'm going to ask you to put it all together, try some new things, and *have fun* with your new skills. The lab comes in two parts 6a and 6b, which you'll get next week, and is due **March 1st** (so two weeks *after* you get part b).

In part a, we'll be playing with APIs - namely the ArcGIS API and Twitter's Streaming API. We use Esri's because it is the direction they are taking their product and Twitter's because it is well established and well-documented. *Their use here is not an endorsement or signal of the meaning to be found in tweets or the mapping thereof.*

### First, let's set up our environment

Ultimately, you are going to make some interactive web maps of tweets for me. However, the exact means by which you do it is left up to you (i.e. will you use Folium or the ArcGIS API, perhaps geopandas to handle the back end, etc.). As such, I'm going to recommend you create an environment with the following libraries, but you may not end up using them all:

`conda create -n lab6 python=3.6`

After you've created and activated that environment, install the following:
`conda install -c esri arcgis`

`conda install -c conda-forge geopandas jupyter folium fiona tweepy geopy`

You should recognize most of those libraries, but the last two are new. I am using [tweepy](http://www.tweepy.org/) to access the Twitter API, there are lots of other libraries. A popular one you might use is [TwitterSearch](https://pypi.python.org/pypi/TwitterSearch/). Additionally, [geopy](https://pypi.python.org/pypi/TwitterSearch/) is a geocoding library for python. Again, lots of these exist and you are free to find one that works best for you; however, your examples will be using these libraries.

### Now, let's get the required authorizations to use the Twitter API

First, you need a Twitter account. You don't have to ever use it outside of this class and I am *definitively not endorsing Twitter in any way*. They have a free, well-documented API that you can use, that's it.

Once you have an account, [go here](https://apps.twitter.com/) and create a new application. Fill in the appropriate information, for a 'website' link to your github account. You can ignore the Callback URL for this use.

After you have created the application, page over to your **Keys and Access Tokens**. You are going to need to find: A Consumer Key, a Consumer Secret, an Access Token, and an Access Token Secret.

Your Consumer Key and your Consumer Secret are listed in your **Keys and Access Tokens** section. You can also **create your access token** there as well.

These four codes are a way for Twitter to keep track of who is accessing their API, when, and to do what.

#### Let's see if this worked.


In [ ]:
import tweepy

CK = ''
CS = ''
AK = ''
AS = ''

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

api = tweepy.API(auth)


print(api.user_timeline(id='marxbot1', count=1)) #This simply pulls the last tweet from an account

### If that ran successfully, you should have a giant mess of text.

That's the data that accompanies a single tweet. Interesting, huh? Check out the [reference docs](http://tweepy.readthedocs.io/en/v3.5.0/api.html#) for tweepy and spend some time experimenting if you want.

Here, I'll pull the same tweet as above, but this time I'm **only** going to print out the text property of the Status object and then check for some location information.



In [30]:
#Note: First I specify which list object I want, then I pull a property from it.

print(api.user_timeline(id='marxbot1', count=1)[0].text)

#Now, let's see if there's some lat and long associated with the tweet
print(api.user_timeline(id='marxbot1', count=1)[0].geo)
print(api.user_timeline(id='marxbot1', count=1)[0].coordinates)

But it is not the least idea of the extent of vigilance and attention on the part of the capital-value for the individual capitalist.
None
None


### (Un)fortunately, most tweets don't actually have location information associated with them. 

There's been *a lot* written about this and the numbers vary from under 5% to 20% or so of tweets. Additionally, it's been argued that upwards of 60% of tweets *can* have some location inferred due to language use, topic, etc.

That's all interesting (and please do email me for citations if you so desire); however, it's also kind of besides the point here. We're interested in learning how to interact with APIs and process data, we can argue about the ephemerality of said data another day.

Let's query some topic of interest and see if we can find some spatial data.

Now, it's important to note that Twitter has both a search and a streaming API. So far, we've been using the search API - we search for existing tweets. I am now going to switch to the streaming one; **however**, lab6scratch.py has an example of how to step through search API results.


In [ ]:
import tweepy
#We're going to set up a couple of tricks here

CK = ''
CS = ''
AK = ''
AS = ''

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

#By setting these values to true, our code will automatically wait as it hits its limits
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#Now I'm going to set up a stream listener
class CustomStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        text = status.text
        user = status.author.screen_name
        print('%s tweeted %s' % (user, text))
    
i = 1
while i<50:
    try:
        
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        #This next line puts a bounding box roughly around Seattle/Tacoma.
        #You start in the southwest and then go to the northeast
        #The format is longitude, then latitude... cuz Twitter
        stream.filter(locations=[-122.626, 47.113, -121.754,47.87])
        i += 1
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue
        

In [2]:
import json

#Note: First I specify which list object I want, then I pull a property from it.

x = (api.user_timeline(id='bricker', count=1)[0]._json)


#### PRETTY JSON FORMAT - MUST DUMP WITH SORT KEYS AND DEFINE INDENT SaPCE  ####
j = json.dumps(x, sort_keys=True, indent=4)
print(j)

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Wed Feb 14 05:16:27 +0000 2018",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [],
        "user_mentions": [
            {
                "id": 869709367737634816,
                "id_str": "869709367737634816",
                "indices": [
                    3,
                    11
                ],
                "name": "Daniel Coe",
                "screen_name": "geo_coe"
            }
        ]
    },
    "favorite_count": 0,
    "favorited": false,
    "geo": null,
    "id": 963643075104006145,
    "id_str": "963643075104006145",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "is_quote_status": false,
    "lang": "en",
    "place": null,
    "retweet_count": 4,
    "retweeted": false,
    "retweeted_status": {
        "contributors

### If that all worked, you now have a listener that will pull tweets from a bounded area you define.

**Cool**. Well, I think so. But, even though we're now pulling tweets *from* a location, you aren't saving their spatial data... *quite yet*.

That's where the lab actually begins.

### Question 1: Where the tweets at?

Using the example code above **and** what you'll find in the lab 6 scratchpad, start pulling the spatial information from the tweets in question. Create a 'file' that contains a tweet's author (account name), its text, and the location from which it came (in latitude and longitude). This 'file' can be in a number of formats (geojson, txt, csv, etc.). 

Bear in mind, there are *a few* ways you can pull location information. You can find the [twitter api documentation here](https://developer.twitter.com/en/docs/tutorials/filtering-tweets-by-location).

Some tweets will come from a 'location' that is a named place. In order to handle those, you will need to geocode the information. The function below takes a string and returns latitude and longitude. Start there.

In [ ]:
from geopy import geocoders
import tweepy
import geojson
import json

CK = ''
CS = ''
AK = ''
AS = ''

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

#By setting these values to true, our code will automatically wait as it hits its limits
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


feature_list = []

def geo(location):
    '''geocoder'''
    try:
        g = geocoders.Nominatim() #I use Nominatim, there are many others
        loc = g.geocode(location)
        return loc.latitude, loc.longitude
    except AttributeError:
        #occasionally a non-geocodable place is entered, return 0's in that case
        print("Couldn't return coords")
        return 0.0, 0.0

def checkHelper(my_list):
    '''helper for shutting down stream'''
    if len(my_list) < 5:
        return True
    else:
        return False

class CustomStreamListener(tweepy.StreamListener):
    
    def getProps(self, tweet):
        '''template for the attributes we want to store'''
        props = {"name":tweet['user']['name'],
                 "screen_name":tweet['user']['screen_name'],
                 "location":tweet['user']['location'],
                 "text":tweet['text']}
        return props

    def on_data(self, data):
        ''''''
        info = json.loads(data)
        if info['coordinates']:
            # get coordinates
            lat = info['coordinates']['coordinates'][1]
            lon = info['coordinates']['coordinates'][0]
            # get attributes
            props = self.getProps(info)
            
            # create geojson
            feature = geojson.Feature(geometry=geojson.Point((lon, lat)), properties=props)
            feature_list.append(feature)
            print("feature added", len(feature_list))
            return checkHelper(feature_list)

        elif info['place']:
            #geocode place
            place = info['place']['full_name']
            lat, lon = geo(place)
            
            #create geojson if coords valid
            if lat != 0.0 and lon != 0.0:
                props = self.getProps(info)
                feature = geojson.Feature(geometry=geojson.Point((lon, lat)), properties=props)
                feature_list.append(feature)
                print("feature added", len(feature_list))
                return checkHelper(feature_list)
                
            else:
                print("feature skipped, bad coords")
                pass
        else:
            #geocode user location
            place = info['user']['location']
            lat, lon = geo(place)
            
            #create geojson if coords valid
            if lat != 0.0 and lon != 0.0:
                props = self.getProps(info)
                feature = geojson.Feature(geometry=geojson.Point((lon, lat)), properties=props)
                feature_list.append(feature)
                print("feature added", len(feature_list))
                return checkHelper(feature_list)
                
            else:
                print("feature skipped, bad coords")
                pass

# Need both the len check AND for the helper check to return false 
# to exit while loop & stop the stream for some reason that is unclear to me
# Helper returning false doesn't stop the stream as far as I can tell, must
# disconnect after exiting the while loop.
while len(feature_list) < 5: 
    try:
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        
        #bounding box roughly around Seattle/Tacoma.
        stream.filter(locations=[-122.626, 47.113, -121.754,47.87])
        
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue

#stop stream
stream.disconnect()
print("Stream stopped")

#create geojson feature collection for future use
feature_collection=geojson.FeatureCollection(feature_list)
print("Features:\n", feature_collection)

### Question 2: Tweets on a map.

Now that you have a 'file' (or a script that will extract author, text, and location from tweets), let's make a map.

Using Folium, ArcGIS API for Python, GeoPandas, or Arcpy, create a map from your file. Make sure you accumulate enough tweets (let's say 100 or so) before you create the map.

Next week, we'll get into how to update the map on the fly and make it more interactive; for now, just make sure you can query some tweets, parse the data, put that data into a GIS of some form.

In [ ]:
from geopy import geocoders
import tweepy
import geojson
import json
import folium

CK = ''
CS = ''
AK = ''
AS = ''

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

#By setting these values to true, our code will automatically wait as it hits its limits
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#list to store geojson features
feature_list = []

def geo(location):
    try:
        g = geocoders.Nominatim() #I use Nominatim, there are many others
        loc = g.geocode(location)
        return loc.latitude, loc.longitude
    except AttributeError:
        #print("Couldn't return coords")
        return 0.0, 0.0

def checkHelper(my_list):
    '''helper for shutting down stream'''
    if len(my_list) < 100:
        return True
    else:
        return False
# 
class CustomStreamListener(tweepy.StreamListener):
    
    def getProps(self, tweet):
        props = {"name":tweet['user']['name'],
                 "screen_name":tweet['user']['screen_name'],
                 "location":tweet['user']['location'],
                 "text":tweet['text']}
        return props

    def on_data(self, data):
        ''''''
        info = json.loads(data)
        if info['coordinates']:
            lat = info['coordinates']['coordinates'][1]
            lon = info['coordinates']['coordinates'][0]
            props = self.getProps(info)

            feature = geojson.Feature(geometry=geojson.Point((lon, lat)), properties=props)
            feature_list.append(feature)
            #print("feature added", len(feature_list))
            return checkHelper(feature_list)

        elif info['place']:
            place = info['place']['full_name']
            lat, lon = geo(place)
            
            if lat != 0.0 and lon != 0.0:
                props = self.getProps(info)
                feature = geojson.Feature(geometry=geojson.Point((lon, lat)), properties=props)
                feature_list.append(feature)
                #print("feature added", len(feature_list))
                return checkHelper(feature_list)
                
            else:
                #print("feature skipped, bad coords")
                pass
        else:
            place = info['user']['location']
            lat, lon = geo(place)
            if lat != 0.0 and lon != 0.0:
                props = self.getProps(info)
                feature = geojson.Feature(geometry=geojson.Point((lon, lat)), properties=props)
                feature_list.append(feature)
                #print("feature added", len(feature_list))
                return checkHelper(feature_list)
                
            else:
                #print("feature skipped, bad coords")
                pass

# need both the len check AND for the helper check to return false 
# to exit while loop for some reason that is unclear to me
while len(feature_list) < 100: 
    try:
        stream = tweepy.Stream(auth=api.auth, listener=CustomStreamListener())
        
        #bounding box roughly around Seattle/Tacoma.
        stream.filter(locations=[-122.626, 47.113, -121.754,47.87])
        
        
    except Exception as e:
        print(e)
        print('Trying to continue')
        continue
        

stream.disconnect()
print("Stream stopped")

#create feature collection
feature_collection=geojson.FeatureCollection(feature_list)
print("Features collected:", len(feature_list))

# create folium map
my_map = folium.Map(
                    location=[41, -98],
                    zoom_start=3,
                    tiles='CartoDB Positron'
                    )

# Add geojson line to map. Was not able to figure out how to style the geojson.
# Also, it annoys me that most everthing that can use geojson seems to only want
# a file. 
my_points = folium.GeoJson(feature_collection).add_to(my_map)

my_map

### Bonus: Did I say you could use any 'GIS,' how about all of them?

Create a script that allows the user to specify in what format he would like the resulting map (ArcGIS API, Shapefile, GeoPandas, or Folium) +1 point for each additional format (**total possible bonus: +3**)